In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from word2vec_selection.classes import Word2VecModel
from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data..

In [4]:
save_path = 'grid_search_results_new.json'
grids_path = 'grid_search_grids.json'

In [5]:
from word2vec_selection.functions import *
model_w2v_settings = return_best_model()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [6]:
model_w2v = Word2VecModel(model_w2v_settings)

In [7]:
# read preprocessed data from pickle file
df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

,title,is_clickbait,text,dataset,sample
0,"[seventeen, time, game, throne, matched, perfe...",1,NaN,clickbait-dataset,train
1,"[facing, texas, tar, heel, territory, duke, fi...",0,NaN,clickbait-dataset,train
2,"[several, gop, state, move, block, funding, sa...",0,A number of GOP states have moved to introduce...,clickbait-news-detection,train
3,"[hillary, clinton, john, kerry, divergent, pat...",0,"WASHINGTON — Early in 2011, after a hectic ...",fake-news,train
4,"[wikinews, interview, kevin, baugh, president,...",0,NaN,clickbait-dataset,train


In [8]:
df['sample'].value_counts()


sample
train    66664
test      3703
val1      1852
val2      1852
Name: count, dtype: int64

In [9]:
# split data into train and test stratified by y
from sklearn.preprocessing import StandardScaler
import os
import pickle

df['title_vector'] = [get_word_vectors(model_w2v, title, aggregation='mean') for title in df['title']]
df

,title,is_clickbait,text,dataset,sample,title_vector
0,"[seventeen, time, game, throne, matched, perfe...",1,NaN,clickbait-dataset,train,"[-0.35697305, 0.060773212, -0.14922297, 1.3295..."
1,"[facing, texas, tar, heel, territory, duke, fi...",0,NaN,clickbait-dataset,train,"[0.07802807, -0.04521935, -0.052836318, 0.3131..."
2,"[several, gop, state, move, block, funding, sa...",0,A number of GOP states have moved to introduce...,clickbait-news-detection,train,"[0.08825532, 0.45959032, 0.1676191, -0.0458484..."
3,"[hillary, clinton, john, kerry, divergent, pat...",0,"WASHINGTON — Early in 2011, after a hectic ...",fake-news,train,"[-0.065645576, 0.27219072, -0.105085365, 0.255..."
4,"[wikinews, interview, kevin, baugh, president,...",0,NaN,clickbait-dataset,train,"[1.0316621, 0.07722468, -0.7290555, -0.7217082..."
...,...,...,...,...,...,...
74072,"[modijis, gujarat, run, patience, customer, at...",0,People furious over lack of cash in banks on M...,clickbait-news-detection,val2,"[0.10010443, 0.2872335, 0.087123975, -0.140484..."
74073,"[dont, send, someone, glitter, mail, send, dic...",0,across this and found it incredibly funny.Send...,clickbait-news-detection,val2,"[0.009599092, -0.05357022, 0.56980944, 0.57503..."
74074,"[mariah, carey, ring, two, thousand, seventeen...",0,Mariah Carey’s 2016 ended on a low note as the...,clickbait-news-detection,val2,"[-0.0014032858, 0.033093706, 0.30112994, 0.750..."
74075,"[twoparty, racket, incapable, dealing, present...",1,The ‘Two-Party Racket’ Is Incapable of Dealing...,fake-news,val2,"[0.41200075, 0.21824586, -0.005152484, 0.50953..."


In [10]:
df['title_vector'][0]

array([-0.35697305,  0.06077321, -0.14922297,  1.3295176 ,  0.5311293 ,
        0.31932646,  0.01412213,  0.16840689, -0.8577355 , -0.14037684,
       -0.13358489, -0.2855721 ,  0.13571858, -0.16594379,  1.3013978 ,
        0.71673936,  0.24522454, -0.29494774, -0.8973892 , -0.18755664],
      dtype=float32)

In [11]:
variables_to_drop = get_dimensions_to_drop()
variables_to_drop

[6, 12, 9, 19, 14, 15, 16]

In [12]:
df['title_vector'] = [drop_dimensions_from_vector(vector, variables_to_drop) for vector in df['title_vector']]

In [13]:
df['title_vector'][0]

array([-0.35697305,  0.06077321, -0.14922297,  1.3295176 ,  0.5311293 ,
        0.31932646,  0.16840689, -0.8577355 , -0.13358489, -0.2855721 ,
       -0.16594379, -0.29494774, -0.8973892 ], dtype=float32)

In [14]:
X_train = np.vstack(df[df['sample']=='train']['title_vector'])
X_test = np.vstack(df[df['sample']=='val2']['title_vector'])

y_train = df[df['sample']=='train']['is_clickbait']
y_test = df[df['sample']=='val2']['is_clickbait']

# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [290]:
import json

with open(grids_path, 'r') as f:
    model_settings = json.load(f)

model_settings
grids = model_settings['grid_search_grids']
grids

{'decision_tree_grid': {'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'max_features': ['sqrt', 'log2', None]},
 'catboost_grid': {'iterations': [100, 250, 500],
  'learning_rate': [None, 0.001, 0.01, 0.1, 0.2, 0.3],
  'depth': [3, 4, 5, 6, 7, 8],
  'l2_leaf_reg': [1, 3, 5, 7, 9]},
 'lightgbm_grid': {'n_estimators': [50, 100, 250, 500],
  'learning_rate': [None, 0.01, 0.1, 0.2, 0.3],
  'max_depth': [-1, 3, 4, 5, 6, 7, 8],
  'num_leaves': [15, 31, 62, 93, 124, 155, 187, 217]},
 'xgboost_grid': {'n_estimators': [10, 25, 50, 100, 250],
  'learning_rate': [None, 0.001, 0.01, 0.1, 0.2, 0.3],
  'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'gamma': [0, 0.01, 0.1, 0.5, 1, 5]},
 'random_forest_grid': {'n_estimators': [50, 100, 250, 500],
  'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'max_features': ['sqrt', 'log2', None]}}

In [291]:
models_to_CV = list()

for model_name, grid in grids.items():
    # check if model has already been trained - if 'best_params' exists in grid
    if 'best_params' in grid.keys():
        print(f'Model {model_name} already trained')
        continue
    models_to_CV.append((model_name))
models_to_CV

# remove _grid from model names
models_to_CV = [model_name.replace('_grid', '') for model_name in models_to_CV]
models_to_CV

['decision_tree', 'catboost', 'lightgbm', 'xgboost', 'random_forest']

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [16]:
def return_model_with_param(model_name, param = dict()):
    if model_name == 'decision_tree':
        model = DecisionTreeClassifier(**param)
    elif model_name == 'random_forest':
        model = RandomForestClassifier(**param)
    elif model_name == 'xgboost':
        model = XGBClassifier(**param)
    elif model_name == 'lightgbm':
        model = LGBMClassifier(**param, verbose=-1)
    elif model_name == 'catboost':
        model = CatBoostClassifier(**param,verbose=0)
    return model

In [296]:
models_to_CV = [
    # 'xgboost',
    # 'lightgbm', 
    # 'decision_tree', 
    # 'random_forest',  
    # 'catboost'
]

In [297]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from tqdm.notebook import tqdm

# read existing results
try:
    with open(save_path, 'r') as f:
        grid_search_results = json.load(f)
    print('Loaded existing results')
except:
    grid_search_results = dict()
    print('No existing results found - creating new dict')
for model_name in models_to_CV:
    grid_search_results[model_name] = dict()

for model_name in tqdm(models_to_CV, desc = 'Models'):

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # get grid
    grid = grids[model_name+'_grid']

    # generate all combinations of parameters
    import itertools

    keys, values = zip(*grid.items())
    combinations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # do grid search
    
    for params in combinations_dicts:
        grid_search_results[model_name][str(params)] = dict()

    for params in tqdm(combinations_dicts, desc = 'Grid combinations search for model {}'.format(model_name)):
       
        model = return_model_with_param(model_name, params)
        scores_auc_cv_val = list()
        scores_auc_val2 = list()
        scores_auc_train = list()

        scores_f1_cv_val = list()
        scores_f1_val2 = list()
        scores_f1_train = list()

        for train_index, val_index in kf.split(X_train_scaled, y_train):
            X_train_kf = X_train_scaled[train_index]
            X_val_kf = X_train_scaled[val_index]
            y_train_kf = y_train.iloc[train_index]
            y_val_kf = y_train.iloc[val_index]

            model.fit(X_train_kf, y_train_kf)

            # print(model)

        
            scores_auc_cv_val.append(roc_auc_score(y_val_kf, model.predict_proba(X_val_kf)[:,1]))
            scores_auc_val2.append(roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:,1]))
            scores_auc_train.append(roc_auc_score(y_train_kf, model.predict_proba(X_train_kf)[:,1]))

            y_pred_cv_val = model.predict_proba(X_val_kf)[:,1]
            y_pred_val2 = model.predict_proba(X_test_scaled)[:,1]
            y_pred_train = model.predict_proba(X_train_kf)[:,1]

            # print(y_pred_cv_val[y_pred_cv_val>0.5])

            y_pred_cv_val = np.where(y_pred_cv_val > 0.5, 1, 0)
            y_pred_val2 = np.where(y_pred_val2 > 0.5, 1, 0)
            y_pred_train = np.where(y_pred_train > 0.5, 1, 0)


            
            scores_f1_cv_val.append(f1_score(y_val_kf, model.predict(X_val_kf)))
            scores_f1_val2.append(f1_score(y_test, model.predict(X_test_scaled)))
            scores_f1_train.append(f1_score(y_train_kf, model.predict(X_train_kf)))
            break

        # print(scores_f1_train, scores_f1_cv_val, scores_f1_val2)
        # print(scores_auc_train, scores_auc_cv_val, scores_auc_val2)

        grid_search_results[model_name][str(params)]['mean_train_f1'] = np.mean(scores_f1_train)
        grid_search_results[model_name][str(params)]['mean_cv_val_f1'] = np.mean(scores_f1_cv_val)
        grid_search_results[model_name][str(params)]['mean_val2_f1'] = np.mean(scores_f1_val2)

        grid_search_results[model_name][str(params)]['mean_train_auc'] = np.mean(scores_auc_train)
        grid_search_results[model_name][str(params)]['mean_cv_val_auc'] = np.mean(scores_auc_cv_val)
        grid_search_results[model_name][str(params)]['mean_val2_auc'] = np.mean(scores_auc_val2)

        grid_search_results[model_name][str(params)]['f1_diff_cv_val'] = np.mean(scores_f1_train) - np.mean(scores_f1_cv_val)
        grid_search_results[model_name][str(params)]['auc_diff_cv_val'] = np.mean(scores_auc_train) - np.mean(scores_auc_cv_val)

        grid_search_results[model_name][str(params)]['f1_diff_val2'] = np.mean(scores_f1_train) - np.mean(scores_f1_val2)
        grid_search_results[model_name][str(params)]['auc_diff_val2'] = np.mean(scores_auc_train) - np.mean(scores_auc_val2)

        grid_search_results[model_name][str(params)]['params'] = params

        # save results to json file
        with open(save_path, 'w') as f:
            json.dump(grid_search_results, f)
    



    print(grid_search_results[model_name])
    
# grid_search_results

Loaded existing results


Models:   0%|          | 0/2 [00:00<?, ?it/s]

Grid combinations search for model random_forest:   0%|          | 0/84 [00:00<?, ?it/s]

{"{'n_estimators': 50, 'max_depth': None, 'max_features': 'sqrt'}": {'mean_train_f1': 0.9991794631344508, 'mean_cv_val_f1': 0.7476904126462739, 'mean_val2_f1': 0.754605747973471, 'mean_train_auc': 0.9999978386229434, 'mean_cv_val_auc': 0.8819999751578008, 'mean_val2_auc': 0.883870975622567, 'f1_diff_cv_val': 0.25148905048817694, 'auc_diff_cv_val': 0.11799786346514263, 'f1_diff_val2': 0.2445737151609798, 'auc_diff_val2': 0.11612686300037645, 'params': {'n_estimators': 50, 'max_depth': None, 'max_features': 'sqrt'}}, "{'n_estimators': 50, 'max_depth': None, 'max_features': 'log2'}": {'mean_train_f1': 0.9993435551179256, 'mean_cv_val_f1': 0.7514095335725269, 'mean_val2_f1': 0.748339483394834, 'mean_train_auc': 0.9999987761229883, 'mean_cv_val_auc': 0.882886232335966, 'mean_val2_auc': 0.8809485419590627, 'f1_diff_cv_val': 0.24793402154539868, 'auc_diff_cv_val': 0.1171125437870223, 'f1_diff_val2': 0.2510040717230916, 'auc_diff_val2': 0.11905023416392557, 'params': {'n_estimators': 50, 'max_

Grid combinations search for model catboost:   0%|          | 0/540 [00:00<?, ?it/s]

{"{'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 1}": {'mean_train_f1': 0.7062820138538709, 'mean_cv_val_f1': 0.6977488530886589, 'mean_val2_f1': 0.69713400464756, 'mean_train_auc': 0.8585306690117215, 'mean_cv_val_auc': 0.8556698570143942, 'mean_val2_auc': 0.8561796188193735, 'f1_diff_cv_val': 0.008533160765212044, 'auc_diff_cv_val': 0.0028608119973272395, 'f1_diff_val2': 0.00914800920631087, 'auc_diff_val2': 0.0023510501923479366, 'params': {'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 1}}, "{'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 3}": {'mean_train_f1': 0.7064853667605036, 'mean_cv_val_f1': 0.698101130787284, 'mean_val2_f1': 0.6996904024767803, 'mean_train_auc': 0.8588342505692171, 'mean_cv_val_auc': 0.8560709999430505, 'mean_val2_auc': 0.8571568171185598, 'f1_diff_cv_val': 0.008384235973219578, 'auc_diff_cv_val': 0.002763250626166669, 'f1_diff_val2': 0.0067949642837232815, 'auc_diff_val2': 0.001677433450657

In [298]:
# save results
with open(save_path, 'w') as f:
    json.dump(grid_search_results, f)

In [17]:
# load results
with open(save_path, 'r') as f:
    grid_search_results = json.load(f)

In [18]:
df_res= pd.DataFrame()

for model in grid_search_results:
    temp = grid_search_results[model]
    temp = pd.DataFrame.from_dict(temp)
    temp = temp.transpose()
    temp['model'] = model
    df_res = pd.concat([df_res, temp])

In [19]:
df_res = df_res.reset_index()

In [20]:
top_auc = df_res[(df_res['auc_diff_val2']<=0.05)].sort_values(by='mean_val2_auc', ascending=False).head(500)
top_auc

,index,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model
1839,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",0.833564,0.765452,0.753285,0.942467,0.891905,0.893559,0.068113,0.050561,0.08028,0.048908,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",catboost
1833,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",0.81334,0.763993,0.757864,0.929131,0.891228,0.893224,0.049347,0.037903,0.055476,0.035907,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",catboost
1859,"{'iterations': 500, 'learning_rate': 0.2, 'dep...",0.821235,0.762048,0.762111,0.933843,0.888878,0.89313,0.059188,0.044965,0.059124,0.040713,"{'iterations': 500, 'learning_rate': 0.2, 'dep...",catboost
1832,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",0.814093,0.766013,0.762872,0.930168,0.891716,0.892573,0.04808,0.038452,0.051221,0.037595,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",catboost
1834,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",0.812332,0.764977,0.761696,0.928652,0.890821,0.892561,0.047355,0.037832,0.050636,0.036092,"{'iterations': 500, 'learning_rate': 0.1, 'dep...",catboost
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2485,"{'n_estimators': 250, 'learning_rate': None, '...",0.784765,0.760693,0.747093,0.910561,0.886908,0.884897,0.024071,0.023653,0.037672,0.025664,"{'n_estimators': 250, 'learning_rate': None, '...",lightgbm
2486,"{'n_estimators': 250, 'learning_rate': None, '...",0.784765,0.760693,0.747093,0.910561,0.886908,0.884897,0.024071,0.023653,0.037672,0.025664,"{'n_estimators': 250, 'learning_rate': None, '...",lightgbm
2487,"{'n_estimators': 250, 'learning_rate': None, '...",0.784765,0.760693,0.747093,0.910561,0.886908,0.884897,0.024071,0.023653,0.037672,0.025664,"{'n_estimators': 250, 'learning_rate': None, '...",lightgbm
2488,"{'n_estimators': 250, 'learning_rate': None, '...",0.784765,0.760693,0.747093,0.910561,0.886908,0.884897,0.024071,0.023653,0.037672,0.025664,"{'n_estimators': 250, 'learning_rate': None, '...",lightgbm


In [21]:
topf1 = df_res[(df_res['f1_diff_val2']<=0.05)].sort_values(by='mean_val2_f1', ascending=False).head(500)
topf1

,index,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model
1676,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",0.794185,0.759501,0.763425,0.916635,0.887694,0.891328,0.034684,0.028941,0.03076,0.025307,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost
2123,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",0.810769,0.753974,0.761281,0.928892,0.885109,0.886011,0.056795,0.043783,0.049488,0.042881,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",lightgbm
1677,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",0.793948,0.760733,0.760462,0.916067,0.888511,0.890042,0.033215,0.027555,0.033486,0.026025,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost
1956,"{'n_estimators': 50, 'learning_rate': None, 'm...",0.800882,0.755928,0.760379,0.923317,0.885941,0.887859,0.044954,0.037376,0.040503,0.035458,"{'n_estimators': 50, 'learning_rate': None, 'm...",lightgbm
2068,"{'n_estimators': 50, 'learning_rate': 0.1, 'ma...",0.800882,0.755928,0.760379,0.923317,0.885941,0.887859,0.044954,0.037376,0.040503,0.035458,"{'n_estimators': 50, 'learning_rate': 0.1, 'ma...",lightgbm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,"{'iterations': 500, 'learning_rate': None, 'de...",0.758481,0.748019,0.743911,0.890414,0.882337,0.879878,0.010463,0.008076,0.01457,0.010536,"{'iterations': 500, 'learning_rate': None, 'de...",catboost
1731,"{'iterations': 500, 'learning_rate': None, 'de...",0.759155,0.749721,0.743911,0.890535,0.882314,0.880111,0.009434,0.008222,0.015244,0.010424,"{'iterations': 500, 'learning_rate': None, 'de...",catboost
693,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",0.778455,0.753161,0.743849,0.906129,0.884246,0.884195,0.025294,0.021883,0.034606,0.021934,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",xgboost
1666,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",0.769408,0.757029,0.743777,0.897901,0.88497,0.883773,0.012379,0.012931,0.025631,0.014128,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost


In [22]:
top_both = pd.merge(top_auc, topf1, on=['index'], how='inner')

# keep _x columns
top_both = top_both[[col for col in top_both.columns if '_x' in col]]
top_both.columns = [col.replace('_x', '') for col in top_both.columns]

In [23]:
top_both['sum_diff_val2'] = top_both['f1_diff_val2'] + top_both['auc_diff_val2']
top_both['sum_auc_f1_val2'] = top_both['mean_val2_f1'] + top_both['mean_val2_auc']

top_both = top_both.sort_values(by='sum_auc_f1_val2', ascending=False).head(1000).reset_index(drop=True)
top_both

,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model,sum_diff_val2,sum_auc_f1_val2
0,0.794185,0.759501,0.763425,0.916635,0.887694,0.891328,0.034684,0.028941,0.03076,0.025307,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost,0.056067,1.654753
1,0.78874,0.761143,0.759071,0.912354,0.887864,0.892015,0.027597,0.02449,0.029669,0.020339,"{'iterations': 250, 'learning_rate': 0.3, 'dep...",catboost,0.050008,1.651086
2,0.793948,0.760733,0.760462,0.916067,0.888511,0.890042,0.033215,0.027555,0.033486,0.026025,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost,0.05951,1.650504
3,0.793494,0.761149,0.758721,0.915905,0.889569,0.891434,0.032345,0.026336,0.034773,0.024471,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost,0.059244,1.650155
4,0.800577,0.758676,0.759971,0.920349,0.888325,0.889341,0.041901,0.032024,0.040606,0.031008,"{'iterations': 500, 'learning_rate': 0.2, 'dep...",catboost,0.071614,1.649312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,0.789093,0.755835,0.745269,0.913333,0.884204,0.884972,0.033258,0.029129,0.043823,0.028362,"{'n_estimators': 100, 'learning_rate': 0.3, 'm...",lightgbm,0.072185,1.630241
245,0.789093,0.755835,0.745269,0.913333,0.884204,0.884972,0.033258,0.029129,0.043823,0.028362,"{'n_estimators': 100, 'learning_rate': 0.3, 'm...",lightgbm,0.072185,1.630241
246,0.781613,0.755882,0.744727,0.908463,0.885917,0.885483,0.025731,0.022546,0.036885,0.022979,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",xgboost,0.059865,1.630211
247,0.785984,0.756643,0.743982,0.911566,0.886009,0.885835,0.029341,0.025557,0.042001,0.025731,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",lightgbm,0.067733,1.629818


In [24]:
params, model_name = top_both['params'][0], top_both['model'][0]
print('Winner model: |{}|'.format(model_name), 'with params: {}'.format(params))

Winner model: |catboost| with params: {'iterations': 250, 'learning_rate': 0.2, 'depth': 5, 'l2_leaf_reg': 3}


In [25]:
# train model with best params
model = return_model_with_param(model_name, params)
model.fit(X_train_scaled, y_train)

In [ ]:


# save model
import pickle
with open('predictive_models/{}.pkl'.format(model_name), 'wb') as f:
    pickle.dump(model, f)


# save scaler
with open('predictive_models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# calculate metrics on test set
from sklearn.metrics import roc_auc_score, f1_score

X_test = np.vstack(df[df['sample']=='test']['title_vector'])
y_test = df[df['sample']=='test']['is_clickbait']

X_test_scaled = scaler.transform(X_test)

y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:,1]

print('F1 score: {}'.format(f1_score(y_test, y_pred)))
print('AUC score: {}'.format(roc_auc_score(y_test, y_pred_proba)))

F1 score: 0.7570661896243293
AUC score: 0.8819525267854268


In [ ]:
# find best threshold for f1 score
import numpy as np
from tqdm.auto import tqdm
# find threshold for best f1 score
thresholds = np.linspace(0, 1, 10000)
f1s = dict()
for threshold in tqdm(thresholds):
    y_pred = (model.predict_proba(X_test_scaled)[:, 1] >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    f1s[threshold] = f1

# get threshold for best f1 score
threshold = max(f1s, key=f1s.get)
threshold

  0%|          | 0/10000 [00:00<?, ?it/s]

0.34093409340934094

In [ ]:
# find auc and f1 for best threshold
y_pred = (model.predict_proba(X_test_scaled)[:, 1] >= threshold).astype(int)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)

print(f'ROC AUC test: {roc_auc}')
print(f'F1 test: {f1}')

ROC AUC test: 0.8819525267854268
F1 test: 0.7621758380771663


In [ ]:
# find auc and f1 for best threshold on train
y_pred = (model.predict_proba(X_train_scaled)[:, 1] >= threshold).astype(int)
y_proba = model.predict_proba(X_train_scaled)[:, 1]

roc_auc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

print(f'ROC AUC train: {roc_auc}')
print(f'F1 train: {f1}')

ROC AUC train: 0.9123437942996926
F1 train: 0.7942776404097492


In [ ]:
# save threshold to txt file
with open('predictive_models/threshold.txt', 'w') as f:
    f.write(str(threshold))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [38]:
%run -i 'functions.py'
sample = 'Trump ordered to pay New York Times $392,638 in legal fees for failed lawsuit'
predict_on_text(model, model_w2v, sample)




[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package words is already up-to-date!


array([[0.92746605, 0.07253395]])

In [ ]:
model.pre